<a href="https://colab.research.google.com/github/anikaha/OCR_7/blob/main/XMLtoCSV.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
# Import all library
import pandas as pd
import glob
import xml.etree.ElementTree as ET

In [16]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
!unzip '/content/drive/MyDrive/OCR.zip' -d '/tmp/OCR'

Archive:  /content/drive/MyDrive/OCR.zip
  inflating: /tmp/OCR/ImageAndXML_Data/0000223278.tif  
  inflating: /tmp/OCR/ImageAndXML_Data/0000223278_gt.xml  
  inflating: /tmp/OCR/ImageAndXML_Data/0000223278_ocr.xml  
  inflating: /tmp/OCR/ImageAndXML_Data/0001136521.tif  
  inflating: /tmp/OCR/ImageAndXML_Data/0001136521_gt.xml  
  inflating: /tmp/OCR/ImageAndXML_Data/0001136521_ocr.xml  
  inflating: /tmp/OCR/ImageAndXML_Data/0001139626.tif  
  inflating: /tmp/OCR/ImageAndXML_Data/0001139626_gt.xml  
  inflating: /tmp/OCR/ImageAndXML_Data/0001139626_ocr.xml  
  inflating: /tmp/OCR/ImageAndXML_Data/0001139716.tif  
  inflating: /tmp/OCR/ImageAndXML_Data/0001139716_gt.xml  
  inflating: /tmp/OCR/ImageAndXML_Data/0001139716_ocr.xml  
  inflating: /tmp/OCR/ImageAndXML_Data/0001144288.tif  
  inflating: /tmp/OCR/ImageAndXML_Data/0001144288_gt.xml  
  inflating: /tmp/OCR/ImageAndXML_Data/0001144288_ocr.xml  
  inflating: /tmp/OCR/ImageAndXML_Data/0001431487.tif  
  inflating: /tmp/OCR/ImageA

In [17]:
# Function that will extract column data for our CSV file
def xml_to_csv(path):
    xml_list = []
    for xml_file in glob.glob(path + '/*.xml'):
        etree = ET.parse(xml_file)
        ns = {'ms' : 'http://schema.primaresearch.org/PAGE/gts/pagecontent/2013-07-15'}
        myroot = etree.getroot()
        page = myroot.find(".//ms:Page", ns)
        for children in list(page):
          if children.find(".//ms:TextLine",ns):
            coord = children.find(".//ms:Coords",ns).get("points")
            for children2 in children.findall(".//ms:Word", ns):
              coord2 = children2.find(".//ms:Coords",ns).get("points")
              conf = children2.find(".//ms:TextEquiv",ns).get("conf")
              unicod = children2.find(".//ms:Unicode",ns).text
              value = (page.get('imageFilename'),
                      int(page.get('imageWidth')),
                      int(page.get('imageHeight')),
                      int(coord.split(" ")[0].split(",")[0]),
                      int(coord.split(" ")[0].split(",")[1]),
                      int(coord.split(" ")[1].split(",")[0]),
                      int(coord.split(" ")[2].split(",")[1]),
                      int(coord2.split(" ")[0].split(",")[0]),
                      int(coord2.split(" ")[0].split(",")[1]),
                      int(coord2.split(" ")[1].split(",")[0]),
                      int(coord2.split(" ")[2].split(",")[1]),
                      conf,
                      unicod
                      )
              xml_list.append(value)
    column_name = ['filename', 'width', 'height', 'xmin', 'ymin', 'xmax', 'ymax', 'xmin2', 'ymin2', 'xmax2', 'ymax2', 'conf', 'unicode']
    xml_df = pd.DataFrame(xml_list, columns=column_name)
    return xml_df

In [18]:
data_path = '/tmp/OCR/ImageAndXML_Data/'
# apply xml_to_csv() function to convert all XML files in images/ folder into labels.csv
labels_df = xml_to_csv(data_path)
labels_df.to_csv(('labels.csv'), index=None)
print('Successfully converted xml to csv.')

Successfully converted xml to csv.


In [19]:
labels_df

,filename,width,height,xmin,ymin,xmax,ymax,xmin2,ymin2,xmax2,ymax2,conf,unicode
0,2072860354.tif,754,1000,450,125,557,139,450,126,495,137,0.7142857313,Invoice
1,2072860354.tif,754,1000,450,125,557,139,499,126,555,137,0.7300000191,Number:
2,2072860354.tif,754,1000,574,126,611,140,575,127,609,138,0.6639999747,10050
3,2072860354.tif,754,1000,457,171,636,234,474,172,519,183,0.5242857337,Invoice
4,2072860354.tif,754,1000,457,171,636,234,523,172,555,183,0.4539999962,Date:
...,...,...,...,...,...,...,...,...,...,...,...,...,...
65365,2030403394_2030403397.tif,769,1000,192,839,587,888,493,877,513,884,0.2199999988,dan*
65366,2030403394_2030403397.tif,769,1000,192,839,587,888,516,877,523,884,0.2049999982,to
65367,2030403394_2030403397.tif,769,1000,192,839,587,888,526,877,555,886,0.1620000005,monde
65368,2030403394_2030403397.tif,769,1000,192,839,587,888,557,878,581,885,0.34799999,enter
